In [51]:
#!usr/bin/env python3
#! -*- coding: utf-8 -*-
#xiaobaodas version_alpha_w03 2017-03-17 Adding Sorting algorithm
def xiaobaods_w01(date="",category="牛仔裤",length=7,SQL="xiaobaods",choice="热搜核心词",variable="排名",sort=0,fillna="",debug=0,path=""):
    import time
    time_s = time.time()
    import pandas as pd
    import datetime
    import pymysql
    from dateutil.parser import parse
    SQL_msg = {"local":{"host":"127.0.0.1","port":3306,"user":"root","charset":"utf8","passwd":"","db":"baoersqlbasic"},
               "xiaobaods":{"host":"101.201.237.58","port":3306,"user":"program_default","charset":"utf8","passwd":"KQPp5wwZJG33fwFs","db":"baoersqlbasic"}}
    latest_date=datetime.datetime.today().date()-datetime.timedelta(1)
    choice_list = {"热搜修饰词":{"table":"bc_searchwords_hotwords","variable":("搜索人气","相关搜索词数","点击率","点击人气","支付转化率","直通车参考价")},
                  "热搜品牌词":{"table":"bc_searchwords_hotwords","variable":("搜索人气","相关搜索词数","点击率","点击人气","支付转化率","直通车参考价")},
                  "热搜搜索词":{"table":"bc_searchwords_hotwords","variable":("搜索人气","商城点击占比","点击率","点击人气","支付转化率","直通车参考价")},
                  "热搜核心词":{"table":"bc_searchwords_hotwords","variable":("搜索人气","相关搜索词数","点击率","点击人气","支付转化率","直通车参考价")},
                  "热搜长尾词":{"table":"bc_searchwords_hotwords","variable":("搜索人气","商城点击占比","点击率","点击人气","支付转化率","直通车参考价")},
                  "飙升修饰词":{"table":"bc_searchwords_risewords",
                           "variable":("相关搜索词数","搜索人气","词均搜索增长幅度","点击人气","支付转化率","直通车参考价")},
                  "飙升品牌词":{"table":"bc_searchwords_risewords",
                           "variable":("相关搜索词数","搜索人气","词均搜索增长幅度","点击人气","支付转化率","直通车参考价")},
                  "飙升搜索词":{"table":"bc_searchwords_risewords",
                           "variable":("搜索增长幅度","搜索人气","点击率","点击人气","支付转化率","直通车参考价")},
                  "飙升核心词":{"table":"bc_searchwords_risewords",
                           "variable":("相关搜索词数","搜索人气","词均搜索增长幅度","点击人气","支付转化率","直通车参考价")},
                  "飙升长尾词":{"table":"bc_searchwords_risewords",
                           "variable":("搜索增长幅度","搜索人气","点击率","点击人气","支付转化率","直通车参考价")},}
    if category not in ["牛仔裤","打底裤","休闲裤"]:
        category="牛仔裤"
    if length>30 or length<3:
        length=7
    if SQL not in SQL_msg:
        SQL=SQL_msg[0]
    if choice not in choice_list:
        choice = "热搜修饰词"
    if variable not in choice_list[choice]["variable"]:
        variable="排名"
    if date=="":
        date = latest_date
    else:
        date=parse(date).date()  # 修改日期格式
    # Try to connect with the mysql and back a date which minimum.
    try:
        conn = pymysql.connect(host=SQL_msg[SQL]["host"], port=SQL_msg[SQL]["port"], user=SQL_msg[SQL]["user"],passwd=SQL_msg[SQL]["passwd"],
                               charset=SQL_msg[SQL]["charset"], db=SQL_msg[SQL]["db"])
        cursor = conn.cursor()
        cursor.execute("SELECT min(`日期`),max(`日期`) from "+choice_list[choice]["table"]+" where `类目`='"+category+"' and `字段`='"+choice+"';")
        date_limit = cursor.fetchall()
        date_floor = date_limit[0][0]
        date_ceiling = date_limit[0][1] 
        cursor.close()
        conn.close()
    except Exception as e:
        return e
    if date > latest_date:
        date = latest_date
    if date > date_ceiling:
        date = date_ceiling
    if date < date_floor + datetime.timedelta(length-1):
        date = date_floor + datetime.timedelta(length-1)
    # Main program.
    sql_select_f="SELECT CT.`排名`,CT.`搜索词`"
    for i in range(len(choice_list[choice]["variable"])):
        sql_select_f += ",CT.`"+choice_list[choice]["variable"][i]+"`"
    sql_select_m=""
    for i in range(length):
        sql_select_m += ",MAX(CASE ST.日期 WHEN "+str(date - datetime.timedelta(length-i-1)).replace("-","")+" THEN ST."+variable+" ELSE NULL END) AS `日期："+str(date - datetime.timedelta(length-i-1)).replace("-","")+"` "
    sql_select_e="FROM "+choice_list[choice]["table"]+" AS CT LEFT JOIN "+choice_list[choice]["table"]+" AS ST ON CT.搜索词 = ST.搜索词 WHERE CT.`日期` = "+str(date).replace("-","")+" AND CT.类目 = '"+category+"' AND CT.字段='"+choice+"' AND ST.字段='"+choice+"' AND ST.日期 >= "+str(date - datetime.timedelta(length)).replace("-","")+" AND ST.类目 = '"+category+"' GROUP BY CT.`"+variable+"` ORDER BY CT.`排名`;"
    # read msg from Mysql
    conn = pymysql.connect(host=SQL_msg[SQL]["host"], port=SQL_msg[SQL]["port"], user=SQL_msg[SQL]["user"], passwd=SQL_msg[SQL]["passwd"], charset=SQL_msg[SQL]["charset"], db=SQL_msg[SQL]["db"])
    df = pd.io.sql.read_sql_query(sql_select_f+sql_select_m+sql_select_e,conn)
    conn.close()
    # Order
    if sort not in [0,1,2]:
        sort=0
    if variable=="排名":
        sort_p = 1
        fillna_default=501
    else:
        sort_p = -1
        fillna_default=0

    if sort==0:
        df = df.fillna(fillna)
        df["sort"]=df["排名"]
    elif sort==1:
        import numpy as np
        # Sorting algorithm:均差排序
        df = df.fillna(fillna_default)
        df["sort"]=0
        for i in range(len(df)):
            df.ix[i,"sort"]=(np.mean(df.iloc[i,-length:])-np.mean(df.iloc[i,-3:]))/df.ix[i,"排名"]*sort_p
    elif sort==2:
        import numpy as np
        # Sorting algorithm:均差方排序
        df = df.fillna(fillna_default)
        df["sort"]=0
        for i in range(len(df)):
            df.ix[i,"sort"]=(np.mean(df.iloc[i,-length:])-np.mean(df.iloc[i,-3:]))/df.ix[i,"排名"]**2*sort_p
    df.sort_values("sort",ascending=False,inplace=True)
    # Debug
    if debug not in [1,2,8,9]:
        print(df.to_json(orient="index"))
    elif debug== 1:
        print ("- Running time：%.4f s"%(time.time()-time_s))
        print( "  SQL_choice: %r \n- category: %r \n- length: %r \n- date: %r \n- SQL: %r"%(SQL,category,str(length),str(date),sql_select_f+sql_select_m+sql_select_e))
    elif debug== 2:
        print ("- Running time：%.4f s"%(time.time()-time_s))
        print("- date：%r \n- category：%r \n- length：%r \n- SQL：%r \n- table: %r \n- variable：%r \n- debug：%r \n- path: %r"%(str(date),category,str(length),SQL,choice_list[choice]["table"],variable,debug,path))
    elif debug== 8:
        print ("- Running time：%.4f s"%(time.time()-time_s))
        return df
    elif debug==9:
        import os
        print ("- Running time：%.4f s"%(time.time()-time_s))
        path_default=os.path.join(os.path.expanduser("~"), 'Desktop')
        if not os.path.isdir(path):
            path = path_default
        csv_filename="【数据组】["+choice_list[choice]["table"].split("_")[-1]+"_"+category+"_"+choice+"]"+variable+"_"+datetime.datetime.strftime(date,"%m%d")+"-"+str(length)+"_"+SQL+".csv"
        try:
            df.to_csv(path+"\\"+csv_filename)
            print("> 输出CSV文件：",path,",",csv_filename)
        except Exception as e:
            print("> 输出CSV文件失败，错误原因：",e)
    if __name__ == "__main__":
        try:
            import sys
            argv = sys.argv[1]
            argv = eval(argv)
        except:
            argv ={}

In [52]:
df1 = xiaobaods_w01(choice="热搜长尾词",variable="搜索人气",debug=8,sort=2,length=15)

- Running time：0.7796 s


In [53]:
df1

,排名,搜索词,搜索人气,商城点击占比,点击率,点击人气,支付转化率,直通车参考价,日期：20170302,日期：20170303,...,日期：20170308,日期：20170309,日期：20170310,日期：20170311,日期：20170312,日期：20170313,日期：20170314,日期：20170315,日期：20170316,sort
12,13,哈伦裤女春季2017新款 韩版 学生,18169,0.4319,1.2947,13520,0.0427,0.74,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,18886.0,17773.0,17967.0,18169,36.617357
11,12,乐町官方旗舰店,18798,0.9838,0.8220,8130,0.0071,0.68,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18798,34.811111
27,28,短裙女春2017新款 韩版 百搭,13574,0.3364,0.9947,8992,0.0307,0.78,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13551.0,13574,9.226190
26,27,小脚裤女春秋2017新款 百搭 韩版,13675,0.5934,1.1728,9358,0.0956,0.99,0.0,0.0,...,0.0,0.0,14637.0,17272.0,0.0,0.0,0.0,0.0,13675,0.841884
47,48,哈伦裤女春季 韩版2016 宽松百搭,9404,0.3408,1.2380,6817,0.0458,0.81,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11731.0,0.0,0.0,9404,0.748987
50,51,裤子女2017新款,8971,0.3762,1.1844,6402,0.0478,0.90,0.0,0.0,...,3428.0,4584.0,5089.0,6683.0,7278.0,7389.0,7372.0,8757.0,8971,0.498526
39,40,渔网破洞牛仔裤,10529,0.1835,0.9320,8072,0.0320,0.76,0.0,0.0,...,9464.0,0.0,0.0,12154.0,12180.0,0.0,0.0,10264.0,10529,0.366208
80,81,微喇裤女九分,7051,0.3522,1.4250,5445,0.0521,1.18,0.0,0.0,...,0.0,7173.0,0.0,8200.0,8021.0,0.0,0.0,6807.0,7051,0.220576
101,104,背带裤女2017新款,5906,0.2888,1.2437,4292,0.0309,0.67,0.0,0.0,...,0.0,2883.0,2983.0,0.0,0.0,4160.0,0.0,5712.0,5906,0.210780
70,71,背带裤春秋女 2017新款 韩版学生,7747,0.2959,1.0879,5813,0.0240,0.61,0.0,0.0,...,2693.0,3154.0,3612.0,5387.0,5267.0,4347.0,5201.0,5974.0,7747,0.186973
